In [1]:
import os
import numpy as np
from datasets import Dataset, DatasetDict, load_from_disk
import librosa as lib
import random as rd
import pickle

from useful_functions import *

c:\Users\marco\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Complete pipeline

In [3]:
spec = preprocess_pipeline(["Grego_chant.wav"])

In [4]:
np.shape(spec)

(2160, 256, 128)

In [5]:
output = []
for e in spec:
    output.append(np.reshape(e,(-1,)))

# Dataset generation

In [6]:
buf_dict = {"X": output}

In [7]:
train_dataset = Dataset.from_dict(buf_dict)
dataset = {
    'train': train_dataset,
}
dataset = DatasetDict(dataset)
path = 'data/hugging_face_dataset/'
os.makedirs(path, exist_ok=True)
dataset.save_to_disk(path)

Saving the dataset (1/1 shards): 100%|██████████| 2160/2160 [00:00<00:00, 6790.52 examples/s]


In [8]:
dataset_dict = DatasetDict.load_from_disk(path)
print(dataset_dict)
dataset_dict['train'].features

DatasetDict({
    train: Dataset({
        features: ['X'],
        num_rows: 2160
    })
})


{'X': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}